## Démarrage avec JupyterHub

##### 1ère Commande

In [ ]:
print("Hello world")

In [ ]:
table = spark.sql("select * from default.basedos")
table.show()

##### Execution d'une requète SQL simple avec Spark

In [ ]:
requete_SQL = "select count(*) from default.basedos where date_chargement=201801"
count_lignes=spark.sql(requete_SQL)
count_lignes.show()

## 1.Creation d'un dataframe spark a partir d'une table depuis DataSmart

##### Creation dataframe

In [ ]:
df_ma_table=spark.table("default.instances")

##### Comptage lignes

In [ ]:
nb_lignes=df_ma_table.count()
print(nb_lignes)

##### Affichage structure table

In [ ]:
colonnes_types=df_ma_table.dtypes
for i in range(len(colonnes_types)):
    print("%s : %s"%(colonnes_types[i][0],colonnes_types[i][1]))

##### Affichage 10 lignes , sur les 10 ere variables

In [ ]:
df_ma_table.select(df_ma_table.columns[0:10]).show(10,False)

##### Affichage des partitions disponibles et de leur volumétrie

In [ ]:
from pyspark.sql.functions import count
df_ma_table.groupBy("date_chargement").agg(count("date_chargement")).show()

## 2.Manipulation d'une table

##### Filtrage

In [ ]:
df_ma_table_filtree=df_ma_table.filter("date_chargement=20180515 and nomenseigne ='CETELEM'")
df_ma_table_filtree.show(1)

##### Selection colonnes et renommage

In [ ]:
from pyspark.sql.functions import col
df_ma_table_2=(df_ma_table_filtree.select(col("NUMAUTO").alias("Contrat"),
                                         col("CODPRODUITALPHASICLID").alias("Produit"),
                                         col("TITDATENAISSANCE").alias("DateNaissance"))
                                 .orderBy('Contrat'))
df_ma_table_2.show(10)

##### Suppression doublon

In [ ]:
df_ma_table_3=df_ma_table_2.distinct().orderBy('Contrat')
df_ma_table_3.show(10)

##### Ajout d'un champs

In [ ]:
from pyspark.sql.functions import substring
from pyspark.sql.types import IntegerType
df_ma_table_4=df_ma_table_3.withColumn("AnneeNaissance", substring("DateNaissance",0,4).cast(IntegerType()))
df_ma_table_4.show(10)

##### Suppression de colonnes

In [ ]:
df_ma_table_5=df_ma_table_4.drop("DateNaissance")
df_ma_table_5.show(10)

##### Statistiques de base sur les colonnes

In [ ]:
df_ma_table_5.describe().show()

## 4.Export csv

#####  Export dataframe Spark vers fichier HDFS - distribué

In [ ]:
from pyspark.sql.functions import row_number,col
from pyspark.sql.window import Window
extrait_df_ma_table_5=(df_ma_table_5.withColumn("rank", row_number().over(Window.orderBy(col("AnneeNaissance"))))
                                .filter("rank < 100"))
                                
extrait_df_ma_table_5.persist().show(10)

In [ ]:
destination="hdfs:///data/TP/EXPORT/lc78355/"
nom_export="extrait_df"
try:
    extrait_df_ma_table_5.coalesce(1).write.option("header", "true").option("delimiter", ";").parquet(destination+nom_export)
except Exception as e:
    print(str(e))

extrait_df_ma_table_5.unpersist()